In [ ]:
import os
from pathlib import Path

import syft_rds as sy

import syft_flwr

In [ ]:
# reset the simulation folder
!just reset-sim

In [ ]:
# EXAMPLE = "embedded-devices"
EXAMPLE = "quickstart-pytorch"

PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
DATASET_DIR = Path("./datasets/").expanduser().absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

print(f"{PROJECT_DIR = }")
print(f"{DATASET_DIR = }")
print(f"{FLWR_PROJECT_PATH = }")
print(f"{SIM_DIR = }")
print(f"{CONFIG_DIR = }")

## Create a local dataset

In [ ]:
if EXAMPLE == "embedded-devices":
    SYFTBOX_DATASET_NAME = "fashionmnist"

elif EXAMPLE == "quickstart-pytorch":
    SYFTBOX_DATASET_NAME = "cifar10"

syft_flwr.utils.save_dataset_to_disk(
    dataset_name=SYFTBOX_DATASET_NAME, dataset_dir=DATASET_DIR, num_partitions=2
)

## DO loads dataset

In [ ]:
DO = "yash@openmined.org"
DATASET_NAME = f"{SYFTBOX_DATASET_NAME}_part_1"

!just dump-sim-config {DO}

Print out the config

In [ ]:
DO_CONFIG = CONFIG_DIR / f"{DO}.json"
!cat $DO_CONFIG

In [ ]:
do = sy.init_session(host=DO, syftbox_client_config_path=DO_CONFIG)
assert do.host == DO
assert do.email == DO
assert do.is_admin

In [ ]:
DATASET_PRIVATE_PATH = DATASET_DIR / SYFTBOX_DATASET_NAME / DATASET_NAME
DATASET_MOCK_PATH = DATASET_DIR / SYFTBOX_DATASET_NAME / DATASET_NAME
README_PATH = FLWR_PROJECT_PATH / "README.md"

try:
    dataset = do.dataset.create(
        name=SYFTBOX_DATASET_NAME,
        summary=f"Partitioned {SYFTBOX_DATASET_NAME} dataset.",
        description_path=README_PATH,
        path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
    )
    dataset.describe()
except Exception as e:
    print(f"Error: {e}")

In [ ]:
print(do.datasets)

In [ ]:
flwr_project_path = str(FLWR_PROJECT_PATH)
mock_path = str(do.dataset.get(name=SYFTBOX_DATASET_NAME).get_mock_path())

## Run syft_flwr simulation
This can be run after the DS bootstrap the `flwr` project in the `ds.ipynb` notebook

In [ ]:
if (FLWR_PROJECT_PATH / "main.py").exists():
    !just run-sim-client {flwr_project_path} {DO} {mock_path}
else:
    print("main.py does not exist in the project")
    print(
        "Please wait for the DS to bootstrap the project and then come back to run this cell"
    )